<a href="https://colab.research.google.com/github/dnhshl/cc-ai/blob/main/ganganalyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hochladen von Videos**


*   Links auf das Ordnersymbol klicken, dann öffnet sich ein Reiter "Dateien"
*   Mit einem Klick auf das erste Symbol in der Reihe (das Rechteck mit dem Pfeil drin), können Videos hochgeladen werden
*   Nachdem das Video hochgeladen ist und der Namen unten im Code geändert wurde, kann über *Laufzeit* --> *Alle ausführen* der Code ausgeführt werden und im Anschluss das neue Video, welches links in den Reiter 'Dateien' erscheint heruntergeladen werden





**Mediapipe und Bibliotheken installieren**

In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 17.6 MB/s eta 0:00:00


In [2]:
import cv2
import mediapipe as mp
import numpy as np


**Funktion für die Berechnung des Gelenkwinkels**

In [3]:
def calculate_angle(a, b, c):
    a, b, c = np.array([a, b, c])
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180.0 else 360 - angle


In [6]:
def process_video(video_in_filename, video_out_filename="output.mp4", data_filename="Winkel.txt"):
  # input Video einlesen
  video_in = cv2.VideoCapture(video_in_filename)

  # Mediapipe pose detection vorbereiten
  mp_drawing = mp.solutions.drawing_utils
  mp_pose = mp.solutions.pose
  pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

  # output Video vorbereiten       
  frame_width = int(video_in.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(video_in.get(cv2.CAP_PROP_FRAME_HEIGHT))
  size = (frame_width, frame_height)
  frames_per_second = video_in.get(cv2.CAP_PROP_FPS)
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  video_out = cv2.VideoWriter(video_out_filename, fourcc, frames_per_second, size, isColor=True)


  with open(data_filename, 'w') as angle_file :
      while True:
          # lies das nächste Bild vom Input Video
          ret, image = video_in.read()
          # Abbruch am Ende des Videos
          if not ret:
              break

          # Make detection
          results = pose.process(image)

          # Extract landmarks
          # Der try-exept Konstrukt stellt sicher, dass das Video weiterbearbeitet
          # wird, auch wenn keine Landmarks im Bild gefunden wurden. 
          try:
              landmarks = results.pose_landmarks.landmark

              # Get coordinates
              hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])
              knee = np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y])
              ankle_l = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])

              # Calculate angle
              angle = round(calculate_angle(hip, knee, ankle_l), 2)
              #print(f'Winkel im linken Knie: {angle}')
              # hier muss auch ein Timecode oder eine Frame Nummer mitgeschrieben werden
              angle_file.write(f'{angle}, hip, knee, ankle_l\n')
          except:
              # Mache weiter, auch wenn ein Fehler aufgetreten ist.
              pass

          # Render detections
          mp_drawing.draw_landmarks(
              image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
              mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2), 
              mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
          )

          # Save image to output video
          video_out.write(image)


  video_in.release()
  video_out.release()



Um ein Video zu verarbeiten, muss nur die Funktion process_video aufgerufen werden






In [7]:
process_video('seite_pur.mp4', video_out_filename="result.mp4", data_filename="angle.txt")